In [1]:
import logging
import time
from datetime import datetime

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
import dask.config
from distributed import LocalCluster, Client

In [2]:
dask.config.config['distributed']['comm']['timeouts']['connect'] = '300s'
dask.config.config['distributed']['scheduler']['allowed-failures'] = 12
dask.config.config['distributed']['worker']['memory'] = {'target': 0.3, 'spill': 0.5, 'pause': 0.7, 'terminate': 0.95}

In [3]:
n_workers=1
cluster = LocalCluster(n_workers=n_workers, threads_per_worker=6, memory_limit=f'{30E9/n_workers}')#silence_logs=logging.ERROR)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:39355 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 6 Memory: 30.00 GB


In [4]:
df = dd.read_parquet('../data/geohashed_gps_points', engine='pyarrow')
# df = df.partitions[1217:].repartition(npartitions=8).persist()
df.head()

,latitude,longitude,geohash
index,,,
0,-90.0,-39.374215,50h000084212
1,-90.0,-5.624215,5bh000084212
2,-90.0,64.688241,j2n000081208
3,-90.0,90.000000,n00000000000
4,-90.0,98.438280,n0n0000840j8


In [5]:
%%time 
t0 = time.time()
df = df.set_index('geohash')

CPU times: user 3min 6s, sys: 12.5 s, total: 3min 18s
Wall time: 1h 19min 32s


In [11]:
delayed_df = df.to_delayed()

In [ ]:
partitions = [dask.delayed()]

In [13]:
len(delayed_df)

1218

In [8]:
%%time
df.to_parquet('../data/sorted_geohash_pts')

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8788', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/home/alewis/miniconda3/envs/scipy2020/lib/python3.8/site-packages/tornado/websocket.py", line 956, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/home/alewis/miniconda3/envs/scipy2020/lib/python3.8/site-packages/bokeh/server/views/ws.py", line 135, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [9]:
dt_hr = (time.time() - t0)/60/60
with open(f'{datetime.now()}_geohash_sort_time.csv', 'w') as f:
    f.write(f'dt_hr,{dt_hr}')

In [10]:
client.get_worker_logs()

{'tcp://127.0.0.1:33253': (('INFO',
   'distributed.worker - INFO -       Start worker at:      tcp://127.0.0.1:33253'),
  ('INFO',
   'distributed.worker - INFO -          Listening to:      tcp://127.0.0.1:33253'),
  ('INFO',
   'distributed.worker - INFO -          dashboard at:            127.0.0.1:44571'),
  ('INFO',
   'distributed.worker - INFO - Waiting to connect to:      tcp://127.0.0.1:39355'),
  ('INFO',
   'distributed.worker - INFO - -------------------------------------------------'),
  ('INFO',
   'distributed.worker - INFO -               Threads:                          6'),
  ('INFO',
   'distributed.worker - INFO -                Memory:                   30.00 GB'),
  ('INFO',
   'distributed.worker - INFO -       Local Directory: /work/spd-scipy2020/download_manipulate_data/dask-worker-space/worker-nf5umnm8'),
  ('INFO',
   'distributed.worker - INFO - -------------------------------------------------'),
  ('INFO',
   'distributed.worker - INFO -         Register